In [1]:
# tf.__version__ = 1.13.1; np.__version__ = 1.16.3

from game import Game
from player import Player
from player_ai import SharedAI
from constants import *

import numpy as np
import sys

#BUILDING ORDER
# BUILDING_ORDER_ = BUILDING_ORDER + ['DO NOT BUY']

#when in training, you need to set 📌 load=False; 📌 use_max_probability=True
kwargs = {'load':False,
          'name':'',
          'verbose':True,
          'use_max_probability':True,
          'shared_ai':True,
          'game_record_filename':'',
          'prob_mod':0.0}


Using TensorFlow backend.


{'load': False, 'name': '', 'verbose': True, 'use_max_probability': False, 'shared_ai': True, 'game_record_filename': '', 'prob_mod': 0.0}


In [2]:
load, name, verbose = kwargs['load'], kwargs['name'], kwargs['verbose']
use_max_probability = kwargs['use_max_probability']
USE_SHARED = kwargs['shared_ai']

game = Game(0, name=name, options=kwargs)
players = game.players  #the default setting has 4 AI players

"""如果没有game, 就不能初始化, 因为Player需要一个game作为参数。
players = []
for i in range(4)
    players.append(Player(self, i, 'ai'+str(i)))
"""

shared_ai = SharedAI(players)  #我加上去的, 神经网络4个玩家共用, base_player是players[0]

if load:
    players[0].load_ai(USE_SHARED)

"""
game = Game(0, name=name, options=kwargs)
players = game.players

if USE_SHARED:
    shared_ai = SharedAI(players)  #神经网络共用
elif load:
    players[1].load_ai()
    players[2].load_ai()
    players[3].load_ai()
"""
game = Game(0, players, options=kwargs)
game.run()  #先跑一轮, 然后再开始训练

total_turns = []
for k in range(25):  # 25 → 100
    current_cycle = []
    print('---k=%d---' % k)
    for j in range(10):  # 重复 1和2 10次
        sys.stdout.write('in training round j=%d' % j)
        sys.stdout.flush()
        for i in range(50):  #1. 产生50个game的数据
            print(f"now in game{1 + i + 50*j + 10*50*k}")
            new_game = Game(1 + i + 50*j + 10*50*k, players, options=kwargs)
            new_game.run(silent=(not verbose))
            current_cycle.append(new_game.turn)  # 记录turn, 开始估计有100多次, 后面降到67次左右

        sys.stdout.write(' '*30 + '\r')
        print(f"📌 平均turns为{np.mean(np.array(current_cycle))}")
        new_game.train_players()  #2. 用这些数据对神经网络进行训练

    new_game.flush_player_history()

    if USE_SHARED:
        players[0].save_ai()
    else:
        for player in players:
            player.save_ai()
    total_turns.append(current_cycle)
    print('cycle #%d had mean turns of %.01f, sd: %.03f' % (k, np.mean(current_cycle), np.std(current_cycle)))
    print('flushed history')

means = [float(sum(x))/500 for x in total_turns]

with open('machikoro.log','a') as f:
    f.write(name + '\n+++')
    f.write(str(players[0].AI.dice_ai.summary()))
    f.write('+++')
    for i in range(25):
        print('cycle #%d mean: %.01f, sd:%.03f' % (i, means[i], np.std(total_turns[i])))
        f.write('cycle #%d mean: %.01f, sd:%.03f\n' % (i, means[i], np.std(total_turns[i])) )
    f.write('---\n')
print('done!')


note = """
parser.add_argument('--record-game', dest='game_record_filename', help='filename to store a verbal recollection of the game in', type=str, default='')
parser.add_argument('--probability-mod','--prob-mod', dest='prob_mod', type=float,default=0.,
                    help='''standard deviation of factor (mean=1) to multiply probabilitiekwargs = {'load':False,
          'name':'',
          'verbose':True,
          'use_max_probability':False,
          'shared_ai':True,
          'game_record_filename':'',
          'prob_mod':0.0}

#game = Game(0,name='t004',options=kwargs)
#players = game.players
# players[0].load_ai(True)
#shared_ai = SharedAI(players)  #为所有player.AI添加(player.AI.shared属性), 并把所有shared属性设置为SharedAI
print(kwargs)s by for randomized decisionmaking; value < 0.01 recommended''')

"""



Beginning game #0
Player 2, order 1 won in 103 turns
---k=0---
in training round j=0now in game1
Beginning game #1
Player 2, order 2 won in 124 turns
now in game2
Beginning game #2
Player 3, order 2 won in 104 turns
now in game3
Beginning game #3
Player 0, order 1 won in 102 turns
now in game4
Beginning game #4
Player 0, order 2 won in 120 turns
now in game5
Beginning game #5
Player 0, order 3 won in 116 turns
now in game6
Beginning game #6
Player 3, order 0 won in 97 turns
now in game7
Beginning game #7
Player 3, order 2 won in 99 turns
now in game8
Beginning game #8
Player 1, order 3 won in 100 turns
now in game9
Beginning game #9
Player 0, order 2 won in 115 turns
now in game10
Beginning game #10
Player 1, order 1 won in 98 turns
now in game11
Beginning game #11
Player 1, order 3 won in 100 turns
now in game12
Beginning game #12
Player 0, order 3 won in 112 turns
now in game13
Beginning game #13
Player 3, order 1 won in 117 turns
now in game14
Beginning game #14
Player 1, order 1 wo

Player 1, order 2 won in 107 turns
now in game120
Beginning game #120
Player 2, order 2 won in 112 turns
now in game121
Beginning game #121
Player 3, order 0 won in 70 turns
now in game122
Beginning game #122
Player 2, order 3 won in 94 turns
now in game123
Beginning game #123
Player 0, order 1 won in 105 turns
now in game124
Beginning game #124
Player 3, order 3 won in 124 turns
now in game125
Beginning game #125
Player 3, order 0 won in 85 turns
now in game126
Beginning game #126
Player 3, order 1 won in 106 turns
now in game127
Beginning game #127
Player 1, order 3 won in 123 turns
now in game128
Beginning game #128
Player 3, order 2 won in 100 turns
now in game129
Beginning game #129
Player 2, order 0 won in 103 turns
now in game130
Beginning game #130
Player 2, order 0 won in 109 turns
now in game131
Beginning game #131
Player 0, order 0 won in 120 turns
now in game132
Beginning game #132
Player 3, order 2 won in 107 turns
now in game133
Beginning game #133
Player 1, order 3 won i

Player 0, order 1 won in 110 turns
now in game237
Beginning game #237
Player 1, order 1 won in 110 turns
now in game238
Beginning game #238
Player 1, order 3 won in 104 turns
now in game239
Beginning game #239
Player 0, order 1 won in 90 turns
now in game240
Beginning game #240
Player 1, order 2 won in 103 turns
now in game241
Beginning game #241
Player 3, order 2 won in 100 turns
now in game242
Beginning game #242
Player 2, order 3 won in 111 turns
now in game243
Beginning game #243
Player 3, order 0 won in 95 turns
now in game244
Beginning game #244
Player 2, order 2 won in 107 turns
now in game245
Beginning game #245
Player 3, order 0 won in 117 turns
now in game246
Beginning game #246
Player 2, order 0 won in 109 turns
now in game247
Beginning game #247
Player 2, order 2 won in 95 turns
now in game248
Beginning game #248
Player 0, order 2 won in 103 turns
now in game249
Beginning game #249
Player 1, order 0 won in 120 turns
now in game250
Beginning game #250
Player 2, order 1 won i

Player 2, order 1 won in 90 turns
now in game353
Beginning game #353
Player 0, order 1 won in 106 turns
now in game354
Beginning game #354
Player 2, order 0 won in 97 turns
now in game355
Beginning game #355
Player 0, order 0 won in 89 turns
now in game356
Beginning game #356
Player 0, order 2 won in 113 turns
now in game357
Beginning game #357
Player 1, order 1 won in 110 turns
now in game358
Beginning game #358
Player 3, order 0 won in 89 turns
now in game359
Beginning game #359
Player 3, order 1 won in 120 turns
now in game360
Beginning game #360
Player 3, order 0 won in 86 turns
now in game361
Beginning game #361
Player 0, order 0 won in 113 turns
now in game362
Beginning game #362
Player 1, order 3 won in 109 turns
now in game363
Beginning game #363
Player 3, order 1 won in 101 turns
now in game364
Beginning game #364
Player 3, order 3 won in 104 turns
now in game365
Beginning game #365
Player 1, order 3 won in 105 turns
now in game366
Beginning game #366
Player 1, order 1 won in 

Player 0, order 0 won in 102 turns
now in game470
Beginning game #470
Player 3, order 2 won in 103 turns
now in game471
Beginning game #471
Player 3, order 0 won in 101 turns
now in game472
Beginning game #472
Player 2, order 0 won in 105 turns
now in game473
Beginning game #473
Player 2, order 3 won in 113 turns
now in game474
Beginning game #474
Player 0, order 0 won in 101 turns
now in game475
Beginning game #475
Player 3, order 3 won in 88 turns
now in game476
Beginning game #476
Player 0, order 3 won in 120 turns
now in game477
Beginning game #477
Player 0, order 0 won in 90 turns
now in game478
Beginning game #478
Player 2, order 0 won in 104 turns
now in game479
Beginning game #479
Player 1, order 0 won in 107 turns
now in game480
Beginning game #480
Player 1, order 3 won in 93 turns
now in game481
Beginning game #481
Player 3, order 0 won in 106 turns
now in game482
Beginning game #482
Player 3, order 1 won in 109 turns
now in game483
Beginning game #483
Player 0, order 1 won i

...

Player 0, order 1 won in 96 turns
now in game11893
Beginning game #11893
Player 3, order 0 won in 98 turns
now in game11894
Beginning game #11894
Player 3, order 1 won in 80 turns
now in game11895
Beginning game #11895
Player 3, order 1 won in 91 turns
now in game11896
Beginning game #11896
Player 1, order 0 won in 97 turns
now in game11897
Beginning game #11897
Player 2, order 1 won in 78 turns
now in game11898
Beginning game #11898
Player 1, order 2 won in 95 turns
now in game11899
Beginning game #11899
Player 2, order 1 won in 102 turns
now in game11900
Beginning game #11900
Player 3, order 3 won in 98 turns
📌 平均turns为96.3125            
in training round j=8now in game11901
Beginning game #11901
Player 2, order 2 won in 107 turns
now in game11902
Beginning game #11902
Player 2, order 0 won in 81 turns
now in game11903
Beginning game #11903
Player 3, order 2 won in 116 turns
now in game11904
Beginning game #11904
Player 2, order 2 won in 83 turns
now in game11905
Beginning game #119

Player 3, order 3 won in 108 turns
now in game12112
Beginning game #12112
Player 1, order 1 won in 110 turns
now in game12113
Beginning game #12113
Player 0, order 3 won in 116 turns
now in game12114
Beginning game #12114
Player 3, order 2 won in 95 turns
now in game12115
Beginning game #12115
Player 0, order 0 won in 97 turns
now in game12116
Beginning game #12116
Player 2, order 0 won in 85 turns
now in game12117
Beginning game #12117
Player 3, order 0 won in 81 turns
now in game12118
Beginning game #12118
Player 0, order 1 won in 102 turns
now in game12119
Beginning game #12119
Player 2, order 1 won in 90 turns
now in game12120
Beginning game #12120
Player 2, order 2 won in 99 turns
now in game12121
Beginning game #12121
Player 2, order 0 won in 86 turns
now in game12122
Beginning game #12122
Player 0, order 3 won in 108 turns
now in game12123
Beginning game #12123
Player 1, order 0 won in 105 turns
now in game12124
Beginning game #12124
Player 1, order 0 won in 94 turns
now in game

Player 0, order 2 won in 91 turns
now in game12223
Beginning game #12223
Player 1, order 1 won in 87 turns
now in game12224
Beginning game #12224
Player 2, order 1 won in 110 turns
now in game12225
Beginning game #12225
Player 1, order 2 won in 91 turns
now in game12226
Beginning game #12226
Player 1, order 1 won in 99 turns
now in game12227
Beginning game #12227
Player 1, order 1 won in 103 turns
now in game12228
Beginning game #12228
Player 0, order 1 won in 99 turns
now in game12229
Beginning game #12229
Player 0, order 2 won in 115 turns
now in game12230
Beginning game #12230
Player 0, order 2 won in 90 turns
now in game12231
Beginning game #12231
Player 2, order 0 won in 98 turns
now in game12232
Beginning game #12232
Player 1, order 2 won in 100 turns
now in game12233
Beginning game #12233
Player 3, order 1 won in 90 turns
now in game12234
Beginning game #12234
Player 2, order 2 won in 95 turns
now in game12235
Beginning game #12235
Player 1, order 0 won in 109 turns
now in game1

Player 3, order 0 won in 105 turns
now in game12334
Beginning game #12334
Player 2, order 0 won in 101 turns
now in game12335
Beginning game #12335
Player 0, order 0 won in 94 turns
now in game12336
Beginning game #12336
Player 0, order 0 won in 117 turns
now in game12337
Beginning game #12337
Player 1, order 0 won in 113 turns
now in game12338
Beginning game #12338
Player 0, order 0 won in 117 turns
now in game12339
Beginning game #12339
Player 1, order 0 won in 105 turns
now in game12340
Beginning game #12340
Player 2, order 0 won in 109 turns
now in game12341
Beginning game #12341
Player 1, order 0 won in 97 turns
now in game12342
Beginning game #12342
Player 3, order 0 won in 144 turns
now in game12343
Beginning game #12343
Player 0, order 1 won in 102 turns
now in game12344
Beginning game #12344
Player 0, order 1 won in 91 turns
now in game12345
Beginning game #12345
Player 1, order 1 won in 96 turns
now in game12346
Beginning game #12346
Player 2, order 2 won in 113 turns
now in 

Player 1, order 3 won in 81 turns
now in game12444
Beginning game #12444
Player 3, order 0 won in 113 turns
now in game12445
Beginning game #12445
Player 1, order 0 won in 82 turns
now in game12446
Beginning game #12446
Player 1, order 2 won in 67 turns
now in game12447
Beginning game #12447
Player 3, order 0 won in 109 turns
now in game12448
Beginning game #12448
Player 1, order 1 won in 90 turns
now in game12449
Beginning game #12449
Player 1, order 2 won in 116 turns
now in game12450
Beginning game #12450
Player 2, order 2 won in 99 turns
📌 平均turns为98.99111111111111  
in training round j=9now in game12451
Beginning game #12451
Player 1, order 1 won in 98 turns
now in game12452
Beginning game #12452
Player 0, order 0 won in 109 turns
now in game12453
Beginning game #12453
Player 1, order 3 won in 56 turns
now in game12454
Beginning game #12454
Player 0, order 2 won in 103 turns
now in game12455
Beginning game #12455
Player 2, order 2 won in 103 turns
now in game12456
Beginning game #